# 00 - Quickstart

**Purpose**: Introduction to the Ununennium library for satellite imagery machine learning.

This notebook covers:
- Installation verification
- Creating your first GeoTensor
- Basic operations and CRS handling
- Simple model inference

## Installation (Kaggle/Colab)

Run this cell to install the library if running on Kaggle or Google Colab.

In [ ]:
# Uncomment the following line to install ununennium
# !pip install -q ununennium

## Prerequisites and Environment Check

In [ ]:
import sys
import platform

print(f"Python: {sys.version}")
print(f"Platform: {platform.system()} {platform.release()}")

In [ ]:
import torch
import numpy as np

print(f"PyTorch: {torch.__version__}")
print(f"NumPy: {np.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

In [ ]:
import ununennium
print(f"Ununennium: {ununennium.__version__}")

## Reproducibility

In [ ]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Synthetic Data Generation

In [ ]:
from ununennium.core import GeoTensor

# Create synthetic 12-band satellite imagery (Sentinel-2 like)
data = torch.randn(12, 256, 256)  # 12 bands, 256x256 pixels

# Create GeoTensor with CRS information
geotensor = GeoTensor(
    data=data,
    crs="EPSG:32632",  # UTM Zone 32N
    transform=(10.0, 0.0, 500000.0, 0.0, -10.0, 5000000.0),  # 10m resolution
)

print(f"Shape: {geotensor.shape}")
print(f"CRS: {geotensor.crs}")
print(f"Resolution: {geotensor.resolution}")

## Core Workflow: Model Inference

In [ ]:
from ununennium.models import create_model

# Create a U-Net model for segmentation
model = create_model(
    "unet",
    in_channels=12,
    num_classes=5,
    backbone="resnet18",
    pretrained=False,
)
model.eval()

print(f"Model created: {type(model).__name__}")

In [ ]:
# Run inference
with torch.no_grad():
    input_tensor = geotensor.data.unsqueeze(0)  # Add batch dimension
    output = model(input_tensor)

print(f"Input shape: {input_tensor.shape}")
print(f"Output shape: {output.shape}")
print(f"Output classes: {output.shape[1]}")

## Validation Checks

In [ ]:
# Verify shapes
assert geotensor.shape == (12, 256, 256), "GeoTensor shape mismatch"
assert output.shape == (1, 5, 256, 256), "Output shape mismatch"

# Check for NaNs
assert not torch.isnan(output).any(), "Output contains NaNs"
assert not torch.isinf(output).any(), "Output contains Infs"

# Verify CRS preserved
assert geotensor.crs == "EPSG:32632", "CRS not preserved"

print("All validation checks passed.")

## Save Outputs

In [ ]:
from pathlib import Path

ARTIFACT_DIR = Path("artifacts/notebooks/00")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

# Save prediction
torch.save(output, ARTIFACT_DIR / "prediction.pt")
print(f"Saved prediction to {ARTIFACT_DIR / 'prediction.pt'}")

## Next Steps

- **01_geotensor_basics.ipynb**: Deep dive into GeoTensor operations
- **10_spectral_indices.ipynb**: Compute vegetation indices
- **20_training_basics.ipynb**: Train your first model